In [1]:
import os

print("Video Dataset Files:", os.listdir("/kaggle/input/deep-fake-detection-dfd-entire-original-dataset")[:10])
print("Image Dataset Files:", os.listdir("/kaggle/input/deepfake-and-real-images")[:10])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/deep-fake-detection-dfd-entire-original-dataset'

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
print("Complete")

In [ ]:
train_dir="/kaggle/input/deepfake-and-real-images/Dataset/Train"
test_dir="/kaggle/input/deepfake-and-real-images/Dataset/Test"
val_dir="/kaggle/input/deepfake-and-real-images/Dataset/Validation"

#Image Preprocessing
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_generator=datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator=datagen.flow_from_directory(
    val_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

test_generator=datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=32,
    class_mode="binary"
)

In [ ]:
model=Sequential([
    Input(shape=(128,128,3)),
    Conv2D(16,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#Model summary
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3  
)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
model.save("/kaggle/working/deepfake_model.h5")

In [ ]:
import IPython.display as display

output_path='/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences'
video_files = os.listdir(output_path)
print("Available video files:", video_files[:100]) 
video_path = os.path.join(output_path, video_files[4])
display.Video(video_path)

In [ ]:
import cv2

def extract_frames(video_path, output_folder, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    count = 0

    while cap.isOpened() and count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1

    cap.release()
    print(f"Extracted {count} frames from {video_path}")


frames_output_path = "/kaggle/working/extracted_frames"
os.makedirs(frames_output_path, exist_ok=True)

extract_frames(video_path, frames_output_path)

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/deepfake_model.h5")

test_frame = os.path.join(frames_output_path, "frame_0.jpg")
img = image.load_img(test_frame, target_size=(128,128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)


prediction = model.predict(img_array)[0][0]
print("Prediction:", "Fake" if prediction > 0.5 else "Real")